In [1]:
import pandas as pd
import glob
import os
import re

In [3]:
data = pd.read_pickle('/Users/gocchini/Desktop/paper_3/data/data_almost_complete.pkl')
london_postcodes = pd.read_csv('/Users/gocchini/Desktop/paper_3/data/london_postcodes-ons-postcodes-directory-feb22.csv')

In [118]:
data_main_shareholder = pd.concat([pd.read_excel(f, 'Results') for f in glob.glob('/Users/gocchini/Desktop/paper_3/data/FAME_2/*.xlsx')], ignore_index = True)
data_main_shareholder.to_pickle('/Users/gocchini/Desktop/paper_3/data/data_main_shareholder.pkl')

In [2]:
data_main_shareholder = pd.read_pickle('/Users/gocchini/Desktop/paper_3/data/data_main_shareholder.pkl')

In [4]:
def data_processing(data, london_data): 
    data = data.loc[(data['Inactive'] == 'No')]
    data = data[data['Primary UK SIC (2007) code'].notna()]
    data = data[data['Primary trading address'].notna()]
    data = data.drop_duplicates(keep=False)
    data['postcode'] = data['Primary trading address'].apply(lambda x: x.split(',')[-1])
    data['postcode'] = data['postcode'].apply(lambda x: x[1:])
    london_postcodes = london_data['pcd'].tolist()
    london_df = data[data['postcode'].isin(london_postcodes)]
    london_df['Primary UK SIC (2007) code'] = london_df['Primary UK SIC (2007) code'].astype('string')
    return london_df 

In [5]:

london_df = data_processing(data , london_postcodes)

In [7]:
london_intangibles = london_df[london_df['Primary UK SIC (2007) code'].str.contains(
    "^85|^92|^96|^59|^60|^61|^62|^63|^64|^66|^69|^70|^71|^72|^73|^74|^79|^82|^77|^78")]

In [8]:
london_intangible_main_shareholder = london_intangibles.merge(data_main_shareholder, left_on = 'Company name', right_on= 'Company name', how = 'inner')

In [9]:
with open ('/Users/gocchini/Desktop/paper_3/data/list-of-salutations-titles-honorifics.txt') as f: 
    lines = f.readlines()
honorifics = [line.strip() for line in lines]

In [10]:
def natural_entity_finder(entity, honorifics):
    if type(entity) != float:
        if any(honorific in entity for honorific in honorifics): 
            return 'natural person'
        else:
            return 'conglomerate'

In [11]:
london_intangible_main_shareholder['natural_entity_finder'] = london_intangible_main_shareholder.apply(lambda x: natural_entity_finder(x['PSC - Name'], honorifics), axis = 1)
london_intangible_main_shareholder = london_intangible_main_shareholder.loc[london_intangible_main_shareholder['natural_entity_finder'] ==  'natural person']

In [12]:
london_intangible_main_shareholder

,Unnamed: 0_x,Company name,Inactive,Quoted,Branch,OwnData,Woco,R/O Full Postcode,Primary UK SIC (2007) code,Primary trading address,...,"R/O Address, line 1","R/O Address, line 2","R/O Address, line 3",R/O Full Postcode.1,Primary trading address City,postcode,Unnamed: 0_y,CSH - Name,PSC - Name,natural_entity_finder
1,146.0,MORGAN STANLEY GROUP (EUROPE),No,No,No,No,No,E14 4AD,70100.0,"20 Bank Street, Canary Wharf, London, London, ...",...,20 Bank Street,Canary Wharf,NaN,E14 4AD,London,E14 4AD,59997.0,MORGAN STANLEY,Morgan Stanley International Limited,natural person
2,206.0,CHURCHVILLE LIMITED,No,No,No,No,No,CR0 0YN,78200.0,"3rd Floor Suffolk House, George Street, Croydo...",...,3rd Floor Suffolk House,George Street,NaN,CR0 0YN,Croydon,CR0 0YN,60031.0,NaN,Mr Thomas Riccardo White,natural person
5,393.0,VDMS ACTON LIMITED,No,No,No,No,No,W10 6QN,82990.0,"316-318 Latimer Road, London, London, W10 6QN",...,316-318 Latimer Road,NaN,NaN,W10 6QN,London,W10 6QN,60147.0,VISUAL DATA MEDIA SERVICES LLC,Visual Data Media Services Limited,natural person
6,509.0,LAMBERT SMITH HAMPTON LIMITED,No,No,No,No,No,W1T 3PT,70100.0,"55 Wells Street, London, London, W1T 3PT",...,55 Wells Street,NaN,NaN,W1T 3PT,London,W1T 3PT,60216.0,SKIPTON BUILDING SOCIETY,Countrywide Group Limited,natural person
7,728.0,FORTUNE FINANCIAL SERVICES LIMITED,No,No,No,No,No,NW4 4HA,66220.0,"39 Edgeworth Crescent, Hendon, London, London,...",...,39 Edgeworth Crescent,Hendon,NaN,NW4 4HA,London,NW4 4HA,60342.0,NaN,Mr Premji Devraj Halai,natural person
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53587,74820.0,EXCELLENT MEDIA LTD,No,No,No,No,No,W1W 7LT,63120.0,"85 Great Portland Street, London, London, W1W 7LT",...,85 Great Portland Street,NaN,NaN,W1W 7LT,London,W1W 7LT,15618.0,MR MARTIN KALAR,Mr Martin Kalar,natural person
53588,74865.0,PROFIT TRADING LTD,No,No,No,No,No,SW6 7BS,82990.0,"21 Letterstone Road, Fulham, London, London, S...",...,21 Letterstone Road,Fulham,NaN,SW6 7BS,London,SW6 7BS,15645.0,NaN,Mr Tyrell Clement Nicholas,natural person
53589,74873.0,TOTALPLUS SERVICES LTD.,No,No,No,No,No,W1B 5TE,78109.0,"c/o Gpc Financial Management, 423 Linen Hall, ...",...,c/o Gpc Financial Management,423 Linen Hall,NaN,W1B 5TE,London,W1B 5TE,15651.0,MS MARY MAZHINDU,Ms Mary Mazhindu,natural person
53590,74957.0,GOLDSTAR DRY CLEANERS (UK) LIMITED,No,No,No,No,No,E14 7DL,96010.0,"330 Burdett Road, London, London, E14 7DL",...,330 Burdett Road,NaN,NaN,E14 7DL,London,E14 7DL,15691.0,MR ZIAUL HAQUE,Mr Ziaul Haque,natural person


In [83]:
import pickle
websites = london_intangible_main_shareholder['Website'].tolist()
with open('/Users/gocchini/Desktop/paper_3/data/websites_paper_3.pkl', 'wb') as f:
    pickle.dump(websites, f)

# old text

In [40]:
text3 = pd.read_pickle('/Users/gocchini/Desktop/paper_1/scripts_paper_1/df_november_2022.pkl')
text3['actual_url'] = text3['url'].str.replace('http://', '')
text3['actual_url'] = text3['actual_url'].str.replace('https://', '')

In [41]:
london_data = london_intangibles.merge(text3, left_on = 'Website', right_on= 'actual_url')

In [42]:
london_data = london_data.loc[london_data['status']==True]

In [43]:
england_postcodes = pd.read_csv('/Users/gocchini/Downloads/England postcodes.csv')

/var/folders/0f/pzhfvnbj63jg3sk3nj099q3w0000gq/T/ipykernel_13249/2786139566.py:1: DtypeWarning: Columns (18,29,47) have mixed types. Specify dtype option on import or set low_memory=False.
  england_postcodes = pd.read_csv('/Users/gocchini/Downloads/England postcodes.csv')


In [44]:
english_postcodes = england_postcodes['Postcode'].tolist()

In [37]:
def postcode_finder(text):
    postcodes = re.findall(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})', text)
    postcodes = [postcode[1] for postcode in postcodes]
    return postcodes 

In [73]:
london_data['text_postcode'] = london_data['text'].apply(postcode_finder)

In [47]:
# some websites have multiple postcodes, between whiich a fake one. Account for the true one. 
def suspicious_postcodes_finder(postcode, postcode_in_text, english_postcodes):
    if len(postcode_in_text) > 0: 
        if postcode_in_text[0].isupper():
            if postcode_in_text[0] in english_postcodes:
                if postcode != postcode_in_text[0]:
                    return 'SUSPICIOUS!'
                else: 
                    return 'ok'

In [75]:
london_data['suspicious'] = london_data.apply(lambda x: suspicious_postcodes_finder(x['postcode'], x['text_postcode'], english_postcodes), axis = 1)

In [76]:
suspicioius = len(london_data.loc[london_data['suspicious']=='SUSPICIOUS!'])

In [78]:
ok = len(london_data.loc[london_data['suspicious']=='ok'])

# new text

In [51]:
text_3 = pd.read_pickle('/Users/gocchini/Desktop/paper_3/data/scraped_websites_march_23.pkl')
falses = pd.read_pickle('/Users/gocchini/Desktop/paper_3/data/falses.pkl')
text_3 = text_3.loc[text_3['status'] == True]
falses = falses.loc[falses['status'] == True]
dataset = pd.concat([text_3, falses])
dataset['actual_url'] = dataset['url'].str.replace('http://', '')
dataset['actual_url'] = dataset['actual_url'].str.replace('https://', '')

# Rationale behind the processing:

In [55]:
# why didn't I keep the relevant information when scraping the websites? Because I NEVER learn from my mistakes eheheh 
#  here I am merging my websites with the companies reporting the websites 
dataset_merged = dataset.merge(london_intangible_main_shareholder, left_on = 'actual_url', right_on= 'Website', how = 'inner')
# uh oh! multiple companies are reporting the same website! Lets have a look if I can merge it on the companies name now 
full_df = dataset_merged.merge(london_intangible_main_shareholder, left_on = 'Company name', right_on = 'Company name', how = 'inner')
# If I have scraped a website multiple times, lets just keep the first instance 
full_df = full_df.drop_duplicates(subset=['url'], keep='first')
# basically is the same as the websites :) there is some noise but should not be too bad! 

In [57]:
full_df['text_postcodes'] = full_df['text'].apply(postcode_finder)

In [58]:
full_df['suspicious'] = full_df.apply(lambda x: suspicious_postcodes_finder(x['R/O Full Postcode.1_x'], x['text_postcodes'], english_postcodes), axis = 1)

In [60]:
full_df = full_df[['url', 'status', 'text', 'added_urls', 'actual_url', 'Unnamed: 0_x_x',
       'Company name', 'Inactive_x','R/O Full Postcode_x', 'Primary UK SIC (2007) code_x',
       'Primary trading address_x', 'Website_x', 'R/O Address_x',
       'R/O Address, line 1_x', 'R/O Address, line 2_x',
       'R/O Address, line 3_x', 'R/O Full Postcode.1_x',
       'Primary trading address City_x', 'postcode_x', 'Unnamed: 0_y_x',
       'CSH - Name_x', 'PSC - Name_x', 'natural_entity_finder_x','text_postcodes', 'suspicious']]

In [62]:
full_df.to_pickle('/Users/gocchini/Desktop/paper_3/data/complete_df_paper_3.pkl')

In [64]:
sus = len(full_df.loc[full_df['suspicious']=='SUSPICIOUS!'])
not_sus = len(full_df.loc[full_df['suspicious']=='ok'])

In [66]:
sus 

3141

In [67]:
not_sus

3918

In [68]:
not_sus/sus

1.247373447946514

In [ ]:
uguale allo smaller sample! gotta love stats 